1. Disegnare gli alberi tagliati in accordo al parametro di complessità
2. Provate a creare  altri alberi modificando i parametri nel costruttore DecisionTreeClassifier(). Ad esempio, cosa succede se si cambiano i criteri di splitting? Come varia l'accuratezza? Cosa succede all'aumentare della max_depth?

Importo il dataset

In [26]:
import pandas
import matplotlib.pyplot as pyplot
from google.colab import drive

drive.mount('/content/drive/')
df = pandas.read_csv("drive/MyDrive/Machine Learning 2023-2024/titanic.csv", sep=",")
df.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

Imposto le variabili categoriche

In [28]:
df["Survived"] = df["Survived"].astype("category")
df["Pclass"] = df["Pclass"].astype("category")
df["Sex"] = df["Sex"].astype("category")
df["Embarked"] = df["Embarked"].astype("category")
df.dtypes

PassengerId       int64
Survived       category
Pclass         category
Name             object
Sex            category
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin            object
Embarked       category
dtype: object

Creo i set di training e di test

In [29]:
from sklearn.model_selection import train_test_split

feature_names = set(df.columns) - set(["Survived"])
feature_names = list(feature_names)
target_name = "Survived"

X_train, X_test, y_train, y_test = train_test_split(df[feature_names], df[target_name], test_size=0.3, random_state=41) # Fanculo il 42
print(f"Numero di record nel training set: {X_train.shape[0]}")
print(f"Numero di record nel test set: {X_test.shape[0]}")

Numero di record nel training set: 623
Numero di record nel test set: 268


Tolgo le features che non mi serviranno in fase di training

In [30]:
X_train = X_train.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
X_test = X_test.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
X_train.head()

,SibSp,Sex,Pclass,Embarked,Parch,Age,Fare
464,0,male,3,S,0,NaN,8.0500
451,1,male,3,S,0,NaN,19.9667
62,1,male,1,S,0,45.0,83.4750
705,0,male,2,S,0,39.0,26.0000
712,1,male,1,S,0,48.0,52.0000


Converto le variabili categoriche in variabili numeriche
(ma a questo ha avuto senso impostarle come categoriche?)

In [31]:
# TODO: provare a non impostare le variabili categoriche

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for col in ["Sex", "Embarked"]:
    X_train[col] = label_encoder.fit_transform(X_train[col])
    X_test[col] = label_encoder.fit_transform(X_test[col])
X_train.head()

,SibSp,Sex,Pclass,Embarked,Parch,Age,Fare
464,0,1,3,2,0,NaN,8.0500
451,1,1,3,2,0,NaN,19.9667
62,1,1,1,2,0,45.0,83.4750
705,0,1,2,2,0,39.0,26.0000
712,1,1,1,2,0,48.0,52.0000


Sostituisco i valori nulli dell'età con la media dell'età

In [32]:
eta_media = X_train["Age"].mean()
X_train["Age"] = X_train["Age"].fillna(eta_media)
X_test["Age"] = X_test["Age"].fillna(eta_media)
X_train.head()

,SibSp,Sex,Pclass,Embarked,Parch,Age,Fare
464,0,1,3,2,0,29.304165,8.0500
451,1,1,3,2,0,29.304165,19.9667
62,1,1,1,2,0,45.000000,83.4750
705,0,1,2,2,0,39.000000,26.0000
712,1,1,1,2,0,48.000000,52.0000


Converto tutte le colonne in valori numerici

In [33]:
X_train.dtypes

SibSp          int64
Sex            int64
Pclass      category
Embarked       int64
Parch          int64
Age          float64
Fare         float64
dtype: object

In [34]:
for col in X_train.columns:
    X_train[col] = X_train[col].astype(int)
    X_test[col] = X_test[col].astype(int)

Creo il Decision Tree

In [35]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

model = DecisionTreeClassifier(random_state=41) # Ancora fanculo al 42
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=41)

Vario il valore di complessità

In [36]:
path = model.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas
ccp_alphas

array([0.00000000e+00, 1.18024738e-05, 1.98778506e-05, 2.30402359e-05,
       3.56696986e-05, 6.56646724e-05, 6.58517512e-05, 8.23146891e-05,
       1.16737195e-04, 1.60513644e-04, 1.79595685e-04, 2.40770465e-04,
       2.47630023e-04, 2.67522739e-04, 5.35045479e-04, 5.35045479e-04,
       5.35045479e-04, 5.53959049e-04, 6.11480547e-04, 6.68806849e-04,
       7.97423550e-04, 8.02568218e-04, 8.02568218e-04, 8.67641317e-04,
       9.24169463e-04, 9.63081862e-04, 9.63081862e-04, 1.01064146e-03,
       1.01377038e-03, 1.04333868e-03, 1.05411945e-03, 1.07009096e-03,
       1.07009096e-03, 1.20385233e-03, 1.20385233e-03, 1.23060460e-03,
       1.33761370e-03, 1.33761370e-03, 1.37087328e-03, 1.37583123e-03,
       1.42678794e-03, 1.44462279e-03, 1.46821941e-03, 1.47137507e-03,
       1.49934335e-03, 1.51596219e-03, 1.52487961e-03, 1.59036279e-03,
       1.62891624e-03, 1.63075873e-03, 1.71978904e-03, 1.83444164e-03,
       1.86931514e-03, 1.99495529e-03, 2.12052718e-03, 2.14018192e-03,
      

In [37]:
clf_alpha6 = DecisionTreeClassifier(ccp_alpha=ccp_alphas[6])
clf_alpha6.fit(X_train, y_train)
fig, ax = pyplot.subplots(figsize=(150, 100))
plot_tree(clf_alpha6, filled=True, ax=ax)
pyplot.plot()


Output hidden; open in https://colab.research.google.com to view.

In [38]:
clf_alpha9 = DecisionTreeClassifier(ccp_alpha=ccp_alphas[9])
clf_alpha9.fit(X_train, y_train)
fig, ax = pyplot.subplots(figsize=(150, 100))
plot_tree(clf_alpha9, filled=True, ax=ax)
pyplot.plot()


Output hidden; open in https://colab.research.google.com to view.

Vario la max_depth

In [42]:
model_depth_3 = DecisionTreeClassifier(max_depth=3)
model_depth_3.fit(X_train, y_train)
fig, ax = pyplot.subplots(figsize=(150, 100))
plot_tree(model_depth_3, filled=True, ax=ax)
pyplot.plot()

[]

In [ ]:
model_depth_6 = DecisionTreeClassifier(max_depth=6)
model_depth_6.fit(X_train, y_train)
fig, ax = pyplot.subplots(figsize=(150, 100))
plot_tree(model_depth_6, filled=True, ax=ax)
pyplot.plot()

[]

Vario il criterio di splitting

In [41]:
# TODO

Cosa succede all'aumentare della max_depth?
# TODO